# PERSONALITY CLASSIFICATION WITH K-NEAREST NEIGHBOR ALGORITHM
Student Name: Gül Sena ERGUN
Student Number: 2210356113

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib

# Define total number of classes
num_classes = 16

# Create a dict to encode personality labels
label_dict = {'ESTJ': 0, 'ENTJ': 1, 'ESFJ': 2, 'ENFJ': 3, 'ISTJ': 4, 'ISFJ': 5, 'INTJ': 6, 'INFJ': 7, 'ESTP': 8,
              'ESFP': 9, 'ENTP': 10, 'ENFP': 11, 'ISTP': 12, 'ISFP': 13, 'INTP': 14, 'INFP': 15}

def prepare_input_data():
    df = pd.read_csv('16P.csv', encoding='cp1252')
    df = df.drop(['Response Id'], axis=1)
    arr = np.array(df)

    # Independent variables
    predictor = arr[:, :-1]
    # Dependent variable
    target = arr[:, -1]
    # Encode labels
    target = np.array([label_dict[y] for y in target])

    return predictor, target

def KNN(X_train, Y_train, X_test, k):
    y_pred = []
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Loop over query points
    for x in X_test:
        x = x[np.newaxis,:]
        distances = np.sqrt(np.sum(np.square(x - X_train), axis=1))
        k_indices = np.argsort(distances)[:k]
        nearest_labels = Y_train[k_indices]
        most_frequent = np.bincount(nearest_labels).argmax()
        y_pred.append(most_frequent)
    return np.array(y_pred)

def calculate_accuracy(y_true, y_pred):
    num_correct = np.count_nonzero(y_true == y_pred)
    num_total = y_true.size
    accuracy = num_correct/num_total
    return np.array(accuracy)

def calculate_precision(y_true, y_pred):
    precisions = []
    # Loop over classes
    for i in range(num_classes):
        # Find indices of this class in all samples
        class_indices = np.where(y_true == i)[0]
        # Extract predictions for this class
        class_predictions = y_pred[class_indices]
        # Count number of correct predictions
        num_correct = np.count_nonzero(class_predictions == i)
        # Calculate total number of predictions that resulted as this class
        num_total_predicted = np.count_nonzero(y_pred == i)
        # Calculate precision
        precision = num_correct/num_total_predicted
        precisions.append(precision)
    return np.array(precisions)

def calculate_recall(y_true, y_pred):
    recalls = []
    # Loop over classes
    for i in range(num_classes):
        # Find indices of this class in all samples
        class_indices = np.where(y_true == i)[0]
        # Extract predictions for this class
        class_predictions = y_pred[class_indices]
        # Count number of correct predictions
        num_correct = np.count_nonzero(class_predictions == i)
        # Count number of total instances of this class
        num_total_instances = class_indices.size
        # Calculate recall
        recall = num_correct/num_total_instances
        recalls.append(recall)
    return np.array(recalls)

def normalize(predictor):
    min_vector = predictor.min(axis=1)
    min_vector = min_vector.reshape((-1,1))
    max_vector = predictor.max(axis=1)
    max_vector = max_vector.reshape((-1,1))
    denom = max_vector - min_vector
    normalized_features = (predictor - min_vector)/denom
    return normalized_features

def cross_validation(predictor, target, k):
    num_folds = 5
    num_samples = len(predictor)
    indices = [idx for idx in range(num_samples)]
    np.random.shuffle(indices)
    accuracy_sum = 0
    precision_sum = 0
    recall_sum = 0
    for i in range(num_folds):
        print("Running cross validation fold:", i+1)
        start = int(i * num_samples / num_folds)
        # print("start", start)
        end = int((i + 1) * num_samples / num_folds)
        # print("end", end)
        test_indices = indices[start:end]
        # print("test",test_indices)
        # print("Length test:", len(test_indices))
        train_indices = indices[:start] + indices[end:]
        # print("train", train_indices)
        # print("Length train:", len(train_indices))
        X_train, X_test = predictor[train_indices, :], predictor[test_indices, :]
        Y_train, Y_test = target[train_indices], target[test_indices]

        # Run KNN and get the predictions
        Y_pred = KNN(X_train, Y_train, X_test, k)

        # Calculate metrics
        accuracy_fold = calculate_accuracy(Y_test, Y_pred)
        precision_fold = calculate_precision(Y_test, Y_pred)
        recall_fold = calculate_recall(Y_test, Y_pred)
        accuracy_sum += accuracy_fold
        precision_sum += precision_fold
        recall_sum += recall_fold

    # Avarage metrics over folds
    accuracy_avg = accuracy_sum/num_folds
    precision_avg = precision_sum/num_folds
    recall_avg = recall_sum/num_folds
    return accuracy_avg, precision_avg, recall_avg


# Data Preparation(Read-Normalize-Encode Labels)

In [ ]:
# Get input data and labels
predictor,target = prepare_input_data()

# Normalize input data
normalized_predictor = normalize(predictor)

# KNN with Normalized Features(5-Fold Cross Validation)

In [3]:

accuracy_list_normalized = []
precision_list_normalized = []
recall_list_normalized = []

# Do cross validation for different k values using normalized features
for k in [1, 3, 5, 7, 9]:
    accuracy, precision, recall = cross_validation(normalized_predictor, target, k)
    accuracy_list_normalized.append(accuracy)
    precision_list_normalized.append(precision)
    recall_list_normalized.append(recall)

Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5


## Accuracy Metrics with Normalized Features
Accuracy metrics for different k values are presented in this section.

In [7]:
personalities = ['ESTJ', 'ENTJ', 'ESFJ', 'ENFJ', 'ISTJ', 'ISFJ', 'INTJ', 'INFJ', 'ESTP','ESFP', 'ENTP', 'ENFP', 'ISTP', 'ISFP', 'INTP', 'INFP']
k_values = ["k=1", "k=3", "k=5", "k=7", "k=9"]

df = pd.DataFrame(accuracy_list_normalized, columns = ["Accuracy"], index = k_values)
print(df)

     Accuracy
k=1  0.976316
k=3  0.987900
k=5  0.988933
k=7  0.988866
k=9  0.989000


## Precision Metrics with Normalized Features
Precision metrics for different k values are presented in this section. Precision is calculated for each class
separately and results are provided in tabular form.

In [11]:
df = pd.DataFrame(precision_list_normalized, columns = personalities, index = k_values)
print("================================Precision==============================")
print(df)

================================Precision==============================
         ESTJ      ENTJ      ESFJ      ENFJ      ISTJ      ISFJ      INTJ  \
k=1  0.978477  0.977748  0.977357  0.972994  0.977657  0.974584  0.973422   
k=3  0.988840  0.986118  0.985124  0.986164  0.988605  0.986675  0.986108   
k=5  0.990871  0.987716  0.985888  0.987755  0.990128  0.987551  0.988286   
k=7  0.990936  0.987734  0.985796  0.988330  0.990598  0.987490  0.988242   
k=9  0.991487  0.987903  0.985021  0.988272  0.990165  0.987434  0.988123   

         INFJ      ESTP      ESFP      ENTP      ENFP      ISTP      ISFP  \
k=1  0.980047  0.980757  0.977639  0.977361  0.976697  0.981007  0.975902   
k=3  0.990631  0.989915  0.990416  0.989919  0.991429  0.990350  0.988497   
k=5  0.990659  0.990619  0.991180  0.991486  0.991506  0.990902  0.989276   
k=7  0.991150  0.990589  0.990894  0.991192  0.991417  0.990467  0.988215   
k=9  0.991244  0.990908  0.991184  0.990693  0.991488  0.990632  0.989070   

  

## Recall Metrics with Normalized Features
Recall metrics for different k values are presented in this section. Recall is calculated for each class
separately and results are provided in tabular form.

In [12]:
df = pd.DataFrame(recall_list_normalized, columns = personalities, index = k_values)
print("================================Recall==============================")
print(df)

================================Recall==============================
         ESTJ      ENTJ      ESFJ      ENFJ      ISTJ      ISFJ      INTJ  \
k=1  0.977683  0.978113  0.975296  0.980800  0.976329  0.974603  0.977296   
k=3  0.985794  0.990968  0.986961  0.990088  0.988273  0.989848  0.989751   
k=5  0.987562  0.991215  0.988781  0.990429  0.988876  0.990903  0.990664   
k=7  0.987010  0.991173  0.988547  0.990117  0.988568  0.990851  0.990657   
k=9  0.988210  0.991323  0.988496  0.990350  0.988526  0.990933  0.990675   

         INFJ      ESTP      ESFP      ENTP      ENFP      ISTP      ISFP  \
k=1  0.977638  0.981085  0.977437  0.980359  0.978771  0.974723  0.973277   
k=3  0.986489  0.990420  0.983786  0.988624  0.987741  0.986941  0.989133   
k=5  0.988328  0.991486  0.984639  0.988742  0.987679  0.987246  0.989901   
k=7  0.988268  0.991450  0.984589  0.988849  0.987736  0.987971  0.989579   
k=9  0.988803  0.991535  0.984648  0.989119  0.988042  0.986803  0.989848   

     

# KNN without Normalized Features(5-Fold Cross Validation)

In [13]:
accuracy_list = []
precision_list = []
recall_list = []

# Do cross validation for different k values using un-normalized features
for k in [1, 3, 5, 7, 9]:
    accuracy, precision, recall = cross_validation(predictor, target, k)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)

Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5
Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5


## Accuracy Metrics without Normalized Features
Accuracy metrics for different k values are presented in this section.

In [14]:
df = pd.DataFrame(accuracy_list, columns = ["Accuracy(Without Normalization)"], index = k_values)
print(df)

     Accuracy(Without Normalization)
k=1                         0.977533
k=3                         0.988533
k=5                         0.989133
k=7                         0.989333
k=9                         0.989266


## Precision Metrics without Normalized Features
Precision metrics for different k values are presented in this section. Precision is calculated for each class
separately and results are provided in tabular form.

In [15]:
df = pd.DataFrame(precision_list, columns = personalities, index = k_values)
print("================================Precision((Without Normalization)==============================")
print(df)

================================Precision((Without Normalization)==============================
         ESTJ      ENTJ      ESFJ      ENFJ      ISTJ      ISFJ      INTJ  \
k=1  0.981369  0.972602  0.978055  0.977340  0.979504  0.978251  0.973827   
k=3  0.990692  0.985771  0.984295  0.986686  0.989823  0.987569  0.986912   
k=5  0.992042  0.988512  0.986173  0.988050  0.990672  0.987768  0.987233   
k=7  0.991701  0.988572  0.987518  0.988429  0.991482  0.988542  0.987244   
k=9  0.991702  0.988358  0.986953  0.987765  0.990607  0.988253  0.987717   

         INFJ      ESTP      ESFP      ENTP      ENFP      ISTP      ISFP  \
k=1  0.983230  0.980942  0.979882  0.979035  0.978350  0.977677  0.980165   
k=3  0.991760  0.990378  0.990848  0.991539  0.991474  0.990396  0.989304   
k=5  0.991720  0.990698  0.991445  0.991129  0.991346  0.990663  0.989109   
k=7  0.992020  0.991518  0.991186  0.990923  0.991526  0.990372  0.988301   
k=9  0.992535  0.990410  0.991464  0.991424  0.991459  0

## Recall Metrics without Normalized Features
Recall metrics for different k values are presented in this section. Recall is calculated for each class
separately and results are provided in tabular form.

In [16]:
df = pd.DataFrame(recall_list, columns = personalities, index = k_values)
print("================================Recall((Without Normalization)==============================")
print(df)

================================Recall((Without Normalization)==============================
         ESTJ      ENTJ      ESFJ      ENFJ      ISTJ      ISFJ      INTJ  \
k=1  0.980920  0.980068  0.977373  0.980000  0.976617  0.980161  0.979307   
k=3  0.989386  0.991526  0.988501  0.989859  0.989305  0.989884  0.989119   
k=5  0.989343  0.991425  0.989385  0.990411  0.989333  0.991408  0.990120   
k=7  0.989370  0.991394  0.988746  0.990325  0.988316  0.991118  0.990897   
k=9  0.989318  0.991405  0.988717  0.990353  0.988528  0.990514  0.990940   

         INFJ      ESTP      ESFP      ENTP      ENFP      ISTP      ISFP  \
k=1  0.978621  0.979684  0.977400  0.978923  0.980849  0.979222  0.974102   
k=3  0.988612  0.990986  0.984655  0.989031  0.986921  0.987978  0.987475   
k=5  0.988739  0.990236  0.985368  0.988899  0.987520  0.987788  0.989306   
k=7  0.989670  0.991467  0.985963  0.988856  0.988241  0.987784  0.989561   
k=9  0.988859  0.991205  0.985626  0.989091  0.988040  0.98

# Error Analysis for Classification
For the error analysis, I define a function which detects 5 mispredicted samples and returns the feature vectors,
true labels, predicted labels and nearest neighbors for those samples for the analysis. I have used unnormalized
features for this anaysis to make it easier to interpret the features. K value of 9 is used for this analysis. KNN
assumes that instances of same classes would be located closer to each other in space. However, some samples might
violate this assumption. These may be the people who give answers to questions that normally other personality type
people would give. This places these people close to wrong personality type people in our space. KNN can make
wrong predictions in this case.

In [26]:
def error_analysis(predictor, target, k):
    num_samples = len(predictor)
    indices = [idx for idx in range(num_samples)]
    np.random.shuffle(indices)

    test_indices = indices[0:5000]
    train_indices = indices[5000:]
    X_train, X_test = predictor[train_indices, :], predictor[test_indices, :]
    Y_train, Y_test = target[train_indices], target[test_indices]

    # Run KNN and find 5 wrong predictions
    y_wrong_pred = []
    y_true_labels = []
    wrong_pred_features = []
    wrong_pred_nearest_neighbors = []

    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Loop over query points
    i = 0
    for x in X_test:
        x = x[np.newaxis,:]
        distances = np.sqrt(np.sum(np.square(x - X_train), axis=1))
        k_indices = np.argsort(distances)[:k]
        nearest_labels = Y_train[k_indices]
        most_frequent = np.bincount(nearest_labels).argmax()

        if most_frequent != Y_test[i]:
          y_wrong_pred.append(most_frequent)
          y_true_labels.append(Y_test[i])
          wrong_pred_features.append(x)
          wrong_pred_nearest_neighbors.append(nearest_labels)

        i += 1
        if len(y_wrong_pred) == 5:
          break
    return y_wrong_pred, y_true_labels, wrong_pred_features, wrong_pred_nearest_neighbors

wrong_labels, true_labels, features, nearest_labels = error_analysis(normalized_predictor, target, 9)

In [27]:
print("True Labels:", true_labels)
print("Predicted Labels:", wrong_labels)
print("Nearest neighbors for the wrong predictions:", nearest_labels)

True Labels: [10, 9, 3, 0, 6]
Predicted Labels: [2, 2, 14, 5, 2]
Nearest neighbors for the wrong predictions: [array([2, 2, 2, 2, 2, 2, 2, 2, 2]), array([2, 2, 2, 2, 2, 2, 2, 2, 2]), array([14, 14, 14, 14, 14, 14, 14, 14, 14]), array([5, 5, 5, 5, 5, 5, 5, 5, 5]), array([2, 2, 2, 2, 2, 2, 2, 2, 2])]


# BONUS(Weighted KNN)
In weighted KNN, instead of counting each vote as 1, closest neighbor labels have stronger votes which are inversely
proportional to distances. This would result in slightly better performance. Metrics are calculated for k = 7.

In [ ]:
def weighted_KNN(X_train, Y_train, X_test, k):
    y_pred = []
    X_train = X_train.astype(float)
    X_test = X_test.astype(float)

    # Loop over query points
    for x in X_test:
        x = x[np.newaxis,:]
        distances = np.sqrt(np.sum(np.square(x - X_train), axis=1))
        sorted_distances = np.sort(distances)
        # Get the k nearest distances
        nearest_distances = sorted_distances[:k]
        # Get inverse of the nearest distances
        inverse_nearest_distances = 1/nearest_distances
        # Sum of inverses
        sum_inv_distances = np.sum(inverse_nearest_distances)
        # Get the weights by dividing the inverse distances by sum of inverses
        nearest_weights = inverse_nearest_distances/sum_inv_distances
        k_indices = np.argsort(distances)[:k]
        nearest_labels = Y_train[k_indices]
        scores = {}
        nearest_weights_dict = dict(zip(list(nearest_labels),list(nearest_weights)))
        # Calculate total weights for nearest labels
        for label in nearest_labels:
          if label in scores:
            scores[label] += nearest_weights_dict[label]
          else:
            scores[label] = nearest_weights_dict[label]

        # Find the label with the highest weight
        most_voted = max(scores, key=scores.get)
        y_pred.append(most_voted)
    return np.array(y_pred)

def cross_validation_weighted(predictor, target, k):
    num_folds = 5
    num_samples = len(predictor)
    indices = [idx for idx in range(num_samples)]
    np.random.shuffle(indices)
    accuracy_sum = 0
    precision_sum = 0
    recall_sum = 0
    for i in range(num_folds):
        print("Running cross validation fold:", i+1)
        start = int(i * num_samples / num_folds)
        # print("start", start)
        end = int((i + 1) * num_samples / num_folds)
        # print("end", end)
        test_indices = indices[start:end]
        # print("test",test_indices)
        # print("Length test:", len(test_indices))
        train_indices = indices[:start] + indices[end:]
        # print("train", train_indices)
        # print("Length train:", len(train_indices))
        X_train, X_test = predictor[train_indices, :], predictor[test_indices, :]
        Y_train, Y_test = target[train_indices], target[test_indices]

        # Run KNN and get the predictions
        Y_pred = weighted_KNN(X_train, Y_train, X_test, k)

        # Calculate metrics
        accuracy_fold = calculate_accuracy(Y_test, Y_pred)
        precision_fold = calculate_precision(Y_test, Y_pred)
        recall_fold = calculate_recall(Y_test, Y_pred)
        accuracy_sum += accuracy_fold
        precision_sum += precision_fold
        recall_sum += recall_fold

    # Avarage metrics over folds
    accuracy_avg = accuracy_sum/num_folds
    precision_avg = precision_sum/num_folds
    recall_avg = recall_sum/num_folds
    return accuracy_avg, precision_avg, recall_avg


In [34]:
accuracy, precision, recall = cross_validation_weighted(predictor, target, 7)

Running cross validation fold: 1
Running cross validation fold: 2
Running cross validation fold: 3
Running cross validation fold: 4
Running cross validation fold: 5


## Metrics for Weighted KNN

In [37]:
k_values = ["k=7"]
df = pd.DataFrame(accuracy, columns = ["Accuracy(Without Normalization)"], index = k_values)
print(df)

     Accuracy(Without Normalization)
k=7                         0.989367


In [40]:
df = pd.DataFrame(precision, columns = k_values, index = personalities)
print("================================Precision((Without Normalization)==============================")
print(df)

================================Precision((Without Normalization)==============================
           k=7
ESTJ  0.991764
ENTJ  0.988544
ESFJ  0.987183
ENFJ  0.988218
ISTJ  0.991482
ISFJ  0.988677
INTJ  0.987505
INFJ  0.992236
ESTP  0.990959
ESFP  0.991467
ENTP  0.991228
ENFP  0.991458
ISTP  0.990902
ISFP  0.988865
INTP  0.985555
INFP  0.983957


In [42]:
df = pd.DataFrame(recall, columns = k_values, index = personalities)
print("================================Recall(Without Normalization)==============================")
print(df)

================================Recall(Without Normalization)==============================
           k=7
ESTJ  0.989340
ENTJ  0.991460
ESFJ  0.989061
ENFJ  0.990367
ISTJ  0.989101
ISFJ  0.991243
INTJ  0.990663
INFJ  0.988872
ESTP  0.990998
ESFP  0.985339
ENTP  0.989142
ENFP  0.988306
ISTP  0.988260
ISFP  0.990180
INTP  0.987651
INFP  0.990058
